In [33]:
import pandas as pd
from pycaret.classification import setup, compare_models, ensemble_model, evaluate_model, predict_model, save_model, load_model, tune_model

In [34]:
df = pd.read_csv('../../src/output/alunos_final.csv')
df.head(1)

,status,freq_turma,ccr,nome_docente
0,0,88.33,Algoritmos e programação - GEX003,"ANDRESSA SEBBEN (30), GEOMAR ANDRE SCHREINER (30)"


In [35]:
s = setup(df, target='status', 
          iterative_imputation_iters=10, 
          session_id=123,  
          train_size=0.99, 
          remove_outliers=True, 
          outliers_method='iforest',
          categorical_features=['ccr', 'nome_docente'],
          numeric_features=['freq_turma'],
          fold_strategy='stratifiedkfold',
          fold=10,
          fold_shuffle=True,
          n_jobs=4
          )

,Description,Value
0,Session id,123
1,Target,status
2,Target type,Binary
3,Original data shape,"(14398, 4)"
4,Transformed data shape,"(13689, 4)"
5,Transformed train set shape,"(13545, 4)"
6,Transformed test set shape,"(144, 4)"
7,Numeric features,1
8,Categorical features,2
9,Preprocess,True


In [36]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8374,0.8951,0.6515,0.8656,0.7431,0.6276,0.6414,0.4290
gbc,Gradient Boosting Classifier,0.8318,0.8878,0.6342,0.8646,0.7313,0.6132,0.6290,0.5870
et,Extra Trees Classifier,0.8287,0.8655,0.6555,0.8353,0.7344,0.6107,0.6206,0.5040
dt,Decision Tree Classifier,0.8270,0.8589,0.6540,0.8316,0.7319,0.6069,0.6166,0.3510
rf,Random Forest Classifier,0.8268,0.8801,0.6707,0.8173,0.7366,0.6095,0.6162,0.5340
ada,Ada Boost Classifier,0.8265,0.8814,0.6315,0.8498,0.7244,0.6018,0.6160,0.4630
lda,Linear Discriminant Analysis,0.8207,0.8709,0.6000,0.8620,0.7073,0.5840,0.6040,0.3310
knn,K Neighbors Classifier,0.8204,0.8625,0.6761,0.7963,0.7312,0.5977,0.6023,0.4020
lr,Logistic Regression,0.8192,0.8706,0.6410,0.8195,0.7192,0.5888,0.5986,0.7300
ridge,Ridge Classifier,0.8192,0.0000,0.5905,0.8669,0.7023,0.5791,0.6013,0.3200


In [37]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [38]:
prediction_model = tune_model(best_model, 
                              choose_better = True,
                              n_iter = 2)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8142,0.8823,0.5437,0.9032,0.6788,0.5591,0.5948
1,0.8296,0.8894,0.5728,0.9277,0.7083,0.5972,0.6319
2,0.8226,0.8835,0.5417,0.9426,0.6880,0.5764,0.6195
3,0.8268,0.8893,0.5670,0.9241,0.7028,0.5902,0.6253
4,0.8309,0.8848,0.5689,0.9391,0.7086,0.5993,0.6367
5,0.8161,0.8807,0.5379,0.9203,0.6789,0.5622,0.6020
6,0.8358,0.8879,0.6000,0.9169,0.7254,0.6154,0.6436
7,0.8435,0.9110,0.6019,0.9451,0.7355,0.6320,0.6644
8,0.8204,0.8780,0.5573,0.9111,0.6916,0.5750,0.6096


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [39]:
prediction_model = ensemble_model(best_model, 
                                  method = 'Boosting', 
                                  choose_better = True, 
                                  fold = 2)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8326,0.8807,0.6866,0.8208,0.7477,0.6240,0.6295
1,0.8320,0.8856,0.6812,0.8235,0.7456,0.6219,0.6281
Mean,0.8323,0.8832,0.6839,0.8221,0.7467,0.6229,0.6288
Std,0.0003,0.0025,0.0027,0.0013,0.0011,0.0010,0.0007


Original model was better than the ensembled model, hence it will be returned. NOTE: The display metrics are for the ensembled model (not the original one).


In [40]:
values = predict_model(prediction_model)

In [41]:
best_model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=4, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [46]:
save_model(best_model, '../../src/models/LGBMClassifier')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['freq_turma'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['ccr', 'n...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample

# Predição

Caso já tenha executado os passos anteriores, basta executar as celulas abaixo para realizar a predição.

In [47]:
import ipywidgets as widgets
from IPython.display import HTML
import pandas as pd
from pycaret.classification import predict_model, load_model

In [54]:
df = pd.read_csv('../../src/output/alunos_final.csv')
# create a widget box with dropdown menus with the unique values of the column 'ccr' and nome_docente of alunos_final.csv
ccr = widgets.Dropdown(options = df['ccr'].sort_values().unique().tolist(), description='CCR:')
nome_docente = widgets.Dropdown(options = df['nome_docente'].sort_values().unique().tolist(), description='Docente:')
# frequencia = widgets.Dropdown(options = df['freq_turma'].sort_values().unique().tolist())
# frequencia = widgets.BoundedFloatText(value=0.0, min=0.0, max=100.0, step=0.01, description='Frequência:', disabled=False)
frequencia = widgets.FloatSlider(value=0.0, min=0.0, max=100.0, step=0.01, description='Frequência:', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.2f')

# create a widget box
widgets.HBox([ccr, nome_docente, frequencia])

In [65]:
# create a json file with the values of the dropdown menus
input_data = {
    'ccr': ccr.value,
    'nome_docente': nome_docente.value,
    'freq_turma': frequencia.value
}

prediction_model = load_model('../../src/models/LGBMClassifier')

input_data = pd.DataFrame([input_data])

# with open('../src/input/test.json', encoding='utf-8') as f:
#     test = pd.DataFrame(json.load(f), index=[0])
    
new_prediction = predict_model(prediction_model, data=input_data)

# rename the column 'prediction_label' to 'status' and put in index 0 using insert and pop method
new_prediction.insert(0, 'status', new_prediction.pop('prediction_label'))


Transformation Pipeline and Model Successfully Loaded


In [66]:
HTML(new_prediction.to_html(index=False))

status,ccr,nome_docente,freq_turma,prediction_score
0,Cálculo II - GEX392,EDSON RIBEIRO DOS SANTOS (60),100.0,0.6957
